In [1]:
import sqlite3
import cv2
import face_recognition
from tkinter import *
import PIL
from PIL import Image, ImageTk
import numpy as np
import webbrowser
from datetime import datetime
from datetime import date

root=Tk()
root.title("HOME")
root.state("zoomed")

img = Image.open("backg.png")
img = img.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.ANTIALIAS)
image = ImageTk.PhotoImage(img)

#image = PhotoImage(file="backg.png")

#image = image.zoom(50)

label = Label(root, i=image)
label.place(x = 0, y = 0)

conn = sqlite3.connect('meetdatabase.db')
c = conn.cursor()

frame = LabelFrame(root, padx=20, pady=40, bg="black")
frame.pack(padx=40, pady=120)

def teacher():
  teacher_window = Toplevel(root)
  teacher_window.title("Teacher Login")
  teacher_window.state("zoomed")

  label = Label(teacher_window, i=image)
  label.place(x = 0, y = 0)
  frame = LabelFrame(teacher_window, padx=40, pady=40, bg="black")
  frame.pack(padx=40, pady=200)
  login_button = Button(frame, text="Face Login", font=("bold", 20), borderwidth=5, padx=5, pady=5 ,command=t_login)
  login_button.grid(row=2, column=0, columnspan=2, padx=25, pady=25)
    

def student():
  student_window = Toplevel(root)
  student_window.title("Student Signup")
  student_window.state("zoomed")

  label = Label(student_window, i=image)
  label.place(x = 0, y = 0)

  frame = LabelFrame(student_window, padx=20, pady=10, bg="black")
  frame.pack(padx=40, pady=200)

  login_button = Button(frame, text="Face Login", font=("bold", 20),borderwidth=5, padx=5, pady=5,command=login)
  signup_button = Button(frame, text="Signup", font=("bold", 20),borderwidth=5, padx=5, pady=5,command=signup)
  login_button.grid(row=2, column=0, columnspan=2, padx=25, pady=25)
  signup_button.grid(row=3, column=0, columnspan=2, padx=25, pady=25)




teacher_button = Button(frame, text="You Are A Teacher", font=("bold", 20), borderwidth=5, padx=15, pady=15,command=teacher)
student_button = Button(frame, text="You Are A Student", font=("bold", 20),borderwidth=5, padx=15, pady=15,command=student)
teacher_button.grid(row=2, column=0, columnspan=2, padx=25, pady=25)
student_button.grid(row=3, column=0, columnspan=2, padx=25, pady=25)


def signup_append(username, signup_window):
    webcam_s = cv2.VideoCapture(0)
    ret_s, current_s= webcam_s.read()
    append_enc = face_recognition.face_encodings(current_s)[0]
    blobvariable = append_enc.tobytes()
    c.execute("INSERT INTO faces VALUES(:enc, :name)",
		{
		"enc" : blobvariable,
		"name" : username
		}

		)
    conn.commit()
    signup_window.destroy()

def signup():
    signup_window = Toplevel(root)
    signup_window.title("Signup page")
    signup_window.state("zoomed")
    
    label = Label(signup_window, i=image)
    label.place(x = 0, y = 0)
    
    frame = LabelFrame(signup_window, padx=20, pady=20, bg="black")
    frame.pack(padx=40, pady=200)
    username_label = Label(frame, text="USERNAME", font=("bold", 20), bg="black", fg="white", padx=5, pady=5)
    username_entry = Entry(frame, width=30, borderwidth=5)
    Click_s_button = Button(frame, text="Click Image for signup", font=("bold", 10), padx=5, pady=5, borderwidth=5,command=lambda: signup_append(username_entry.get(), signup_window))
    
    username_label.grid(row=0, column=0, sticky="w", padx=25, pady=25)
    username_entry.grid(row=0, column=1, padx=25, pady=25)
    Click_s_button.grid(row=2, column=0, columnspan=2, padx=25, pady=25)

def join(link, Sname):
    webbrowser.open_new(link)
    webcam = cv2.VideoCapture(0)

    c.execute("SELECT name, encodings FROM faces ")
    data = c.fetchall()
    all_enc = []
    all_img = []
    for tup in data:
      enc_blob = tup[1]
      name = tup[0]
      enc = np.frombuffer(enc_blob)
      all_enc.append(enc)
      all_img.append(name)

    all_face_loc=[]
    all_face_enc=[]
    all_face_name=[]

    time_attended = datetime(1970, 1, 1, 0, 0, 0)
    start = datetime.now().time()
    time_lost = datetime(1970, 1, 1, 0, 0, 0)

    original_list = []
    check_list = []
    check = False


    while True:
        ret, current= webcam.read()
        smallf = cv2.resize(current, (0,0), fx=0.25, fy=0.25)
        landmark_list = face_recognition.face_landmarks(smallf)
        #print(landmark_list)
        #print(",")
        if len(original_list)>80:
            if original_list[-40:] == [original_list[-1]] * 40:
                print("MARKED ABSENT: Using Photo")
                break

            elif len(check_list)==len(original_list):
                print("MARKED ABSENT: Using Video")
                break
            elif len(check_list)>0:
                if str(landmark_list)==original_list[len(check_list)]:
                    check_list.append(str(landmark_list))
                else:
                    original_list.extend(check_list)
                    check_list.clear()
            elif len(check_list)==0:
                if str(landmark_list)==original_list[0]:
                    check_list.append(str(landmark_list))
                else:
                    original_list.append(str(landmark_list))
        else:
            original_list.append(str(landmark_list)) 

        all_face_loc = face_recognition.face_locations(smallf,number_of_times_to_upsample=2,model='hog')
        all_face_enc = face_recognition.face_encodings(smallf,all_face_loc)
        all_face_name=[]

        is_face_detected = False

        if len(all_face_enc)>1:
            print("Attendace deduction- MORE THAN ONE FACE IS AVAILABLE")
            is_face_detected = False
        elif len(all_face_enc) == 0:
            print("Attendace deduction- FACE NOT ON SCREEN")
            is_face_detected = False
        else:
            is_face_detected = True



        for current_face_loc, current_face_enc in zip(all_face_loc, all_face_enc):
            top_pos,right_pos,bottom_pos,left_pos= current_face_loc
            top_pos*=4
            right_pos*=4
            bottom_pos*=4
            left_pos*=4
            currentblob = current_face_enc.tobytes()
            current_face_enc = np.frombuffer(currentblob)
            all_matches= face_recognition.compare_faces(all_enc, current_face_enc)
            name_found = "unknown"
            is_face_detected = False
            if True in all_matches:
                findex = all_matches.index(True)
                name_found = all_img[findex]
                is_face_detected = True
            else:
                print("Attendace deduction- UNRECOGNIZED FACE")
            if name_found!=Sname:
                print("Sorry, you are not " + Sname)
            cv2.rectangle(current,(left_pos,top_pos),(right_pos,bottom_pos),(0,0,225),2)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(current,name_found,(left_pos,bottom_pos),font,1,(0,0,255),1)
        cv2.imshow("WEBCAM",current)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    end = datetime.now().time()
    elapsed_time = datetime.combine(date.today(), end) \
         - datetime.combine(date.today(), start)
    time_attended+= elapsed_time
    print(time_attended)
    print("Attendace time :"+time_attended.strftime("%H:%M:%S"))
    webcam.release()
    cv2.destroyAllWindows()    

    c.execute("INSERT INTO attendance VALUES(:sname, :time)",
		{
		"sname" : name_found,
		"time" : time_attended.strftime("%H:%M:%S")
		}

		)
    conn.commit()

    
    
    
def login():
    webcam = cv2.VideoCapture(0)
    ret, current= webcam.read()
    original = PIL.Image.fromarray(current)
    #original = PIL.ImageTk.PhotoImage(original)
    

    # resize the image using the subsample method
    resized_image = original.resize((100, 100), Image.ANTIALIAS)
    original = PIL.ImageTk.PhotoImage(resized_image)

    img_rec = current

    face_loc = face_recognition.face_locations(img_rec, model="hog")
    face_enc = face_recognition.face_encodings(img_rec, face_loc)

    for current_face_loc, current_face_enc in zip(face_loc, face_enc):
        top_pos,right_pos,bottom_pos,left_pos= current_face_loc
        #all_matches= face_recognition.compare_faces(all_enc, current_face_enc)
        #blob = current_face_enc.tobytes()
        c.execute("SELECT name, encodings FROM faces ")
        data = c.fetchall()
        encodings_list = []
        names = []
        for tup in data:
          enc_blob = tup[1]
          name = tup[0]
          enc = np.frombuffer(enc_blob)
          encodings_list.append(enc)
          names.append(name)
        currentblob=current_face_enc.tobytes()
        current_face_enc=np.frombuffer(currentblob)
        all_matches= face_recognition.compare_faces(encodings_list, current_face_enc)
        name_found = "unknown"
        if True in all_matches:
            findex = all_matches.index(True)
            name_found = names[findex]
        if name_found == "unknown":
            print("Unrecognized Face")
        else:
            new_window = Toplevel(root)
            new_window.title("Home")
            new_window.state("zoomed")
            
            label = Label(new_window, i=image)
            label.place(x = 0, y = 0)
            frame = LabelFrame(new_window, padx=20, pady=20, bg="black")
            frame.pack(padx=40, pady=200)
            

            labelnew = Label(frame, text="Welcome "+name_found+",", font=("bold", 20), bg="black", fg="white",padx=20, pady=20)
            labelnew.grid(row=0, column=0, padx=20, pady=5)
            
            LI = Label(frame, image=original)
            LI.image=original
            LI.grid(row=0, column=1)

            c.execute("SELECT * from links")
            data = c.fetchall()

            names=[]
            links=[]
            for tup in data:
              name=tup[0]
              link=tup[1]
              names.append(name)
              links.append(link)
            rown = 1
            for i in range(len(names)):
              name = names[i]
              link = links[i]

              name_label = Label(frame, text=name+" : ", bg="black", fg="white", font=("bold", 15),padx=10, pady=10)
              name_label.grid(row=rown, column=0, padx=20, pady=20)

              link_button = Button(frame, text="JOIN LINK : " + link, padx=10, pady=10, borderwidth=5, command = lambda:join(link, name_found))
              link_button.grid(row=rown, column=1, padx=20, pady=20)

              rown+=1

            

def teacher_append(meet_link,t_name):
    if len(meet_link)==0:
        return
    c.execute("INSERT INTO links VALUES(:tname, :link)",
		{
		"tname" : t_name,
		"link" : meet_link
		}

		)
    conn.commit()

def attendance(tname):
  c.execute("SELECT * FROM attendance")
  data = c.fetchall()
  print(data)

  new_window = Toplevel(root)
  new_window.title("Check Attendance")
  new_window.state("zoomed")

  label = Label(new_window, i=image)
  label.place(x = 0, y = 0)

  frame = LabelFrame(new_window, padx=20, pady=20, bg="black")
  frame.pack(padx=20, pady=50)

  L = Label(frame, text="Attendance for the class of " + tname, bg="black", fg="white", font=("bold", 15),padx=10, pady=10)
  L.grid(row=0, column=0, padx=20, pady=20)

  names = []
  times = []

  for tup in data:
    name = tup[0]
    time = tup[1]
    names.append(name)
    times.append(time)
  rown = 1
  for i in range(len(names)):
    name = names[i]
    time = times[i]

    name_label = Label(frame, text="Attendance of " + name, bg="black", fg="white", font=("bold", 15),padx=2, pady=2)
    name_label.grid(row=rown, column=0, padx=5, pady=10)

    att = Label(frame, text=time, bg="black", fg="white", font=("bold", 15), padx=2, pady=2)
    att.grid(row=rown, column=1, padx=5, pady=10)
    
    T = datetime(1970, 1, 1, 0, 0, 10).strftime("%H:%M:%S")
    
    LT = T.split(":")
    T = int("".join(LT))
    
    Ltime = time.split(":")
    time = int("".join(Ltime))
    
    print(time)
    print(T)
    
    
    
    if time>=T:
        chk = BooleanVar(value=True)
        present = Checkbutton(frame, text="PRESENT",bg="black", fg="white", padx=2, pady=2, variable=chk, onvalue=1)
        present.select()
        present.grid(row=rown, column=2, padx=5, pady=10)
    else:
        chk = BooleanVar(value=False)
        present = Checkbutton(frame, text="ABSENT",bg="black", fg="white", padx=2, pady=2, variable=chk)
        present.grid(row=rown, column=2, padx=5, pady=10)
    

    rown+=1


    
def t_login():
    webcam = cv2.VideoCapture(0)
    ret, current= webcam.read()
    original = PIL.Image.fromarray(current)
    original = PIL.ImageTk.PhotoImage(original)
    
    # add images and make their encodings and add to the list all_enc_t, also add the respective name to all_img_t to register a teacher
    ks_img = face_recognition.load_image_file("khan_sir.jpg")
    ks_enc = face_recognition.face_encodings(ks_img)[0]
    
    all_enc_t = [ks_enc]
    all_img_t = ["Khan Sir"]
    
    img_rec = current

    face_loc = face_recognition.face_locations(img_rec, model="hog")
    face_enc = face_recognition.face_encodings(img_rec, face_loc)

    for current_face_loc, current_face_enc in zip(face_loc, face_enc):
        top_pos,right_pos,bottom_pos,left_pos= current_face_loc
        all_matches= face_recognition.compare_faces(all_enc_t, current_face_enc)
        name_found = "unknown"
        if True in all_matches:
            findex = all_matches.index(True)
            name_found = all_img_t[findex]
        if name_found == "unknown":
            print("Unrecognized Face")
        else:
            new_window = Toplevel(root)
            new_window.title("Teacher Home, "+name_found)
            new_window.state("zoomed")
            label = Label(new_window, i=image)
            label.place(x = 0, y = 0)
            frame = LabelFrame(new_window, padx=20, pady=20, bg="black")
            frame.pack(padx=40, pady=150)
            labelnew = Label(frame, text="Welcome "+name_found+",", bg="black", fg="white",font=("bold", 20))
            labelnew.grid(row=0, column=0, padx=20, pady=20)
            meet_label = Label(frame, text="Meet link : ", bg="black", fg="white", font=("bold", 15))
            meet_entry = Entry(frame, width=30)
            Click_meet_button = Button(frame, text="Send Meet link", font=("bold", 10),padx=5, pady=5, borderwidth=5, command=lambda: teacher_append(meet_entry.get(), name_found))
            meet_label.grid(row=1, column=0, sticky="w", padx=10, pady=20)
            meet_entry.grid(row=1, column=1)
            Click_meet_button.grid(row=2, column=0, columnspan=2)
            attendance_button = Button(frame, text="Check Attendance", font=("bold", 10), padx=5, pady=5, borderwidth=5, command=lambda:attendance(name_found))
            attendance_button.grid(row=3, column=0, columnspan=2, padx=20, pady=20)

root.mainloop()
conn.close()

C:\Users\91971\AppData\Local\Temp\ipykernel_10184\2668961422.py:17: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  img = img.resize((root.winfo_screenwidth(), root.winfo_screenheight()), Image.ANTIALIAS)
C:\Users\91971\AppData\Local\Temp\ipykernel_10184\2668961422.py:231: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.LANCZOS instead.
  resized_image = original.resize((100, 100), Image.ANTIALIAS)
